In [4]:
import time
!pip install selenium
import selenium
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import itertools

In [5]:
class WebTable:
    def __init__(self, webtable):
       self.table = webtable

    def get_row_count(self):
      return len(self.table.find_elements_by_tag_name("tr")) - 1;

    def get_column_count(self):
        return len(self.table.find_elements_by_xpath("//tr[2]/td"));

    def get_table_size(self):
        return {"rows": self.get_row_count(),
                "columns": self.get_column_count()}

    def row_data(self, row_number):
        if(row_number == 0):
            raise Exception("Row number starts from 1")

        row_number = row_number + 1;
        row = self.table.find_elements_by_xpath("//tr["+str(row_number)+"]/td");
        rData = [];
        for webElement in row :
            rData.append(webElement.text);

        return rData;

    def column_data(self, column_number):
        col = self.table.find_elements_by_xpath("//tr/td["+str(column_number)+"]");
        rData = [];
        for webElement in col :
            rData.append(webElement.text);
        return rData;

    def get_all_data(self):
        # get number of rows
        noOfRows = len(self.table.find_elements_by_xpath("//tr")) -1;
        # get number of columns
        noOfColumns = len(self.table.find_elements_by_xpath("//tr[2]/td"));
        allData = [];
        # iterate over the rows, to ignore the headers we have started the i with '1'
        for i in range(2, noOfRows):
            # reset the row data every time
            ro = [];
            # iterate over columns
            for j in range(1, noOfColumns) :
                # get text from the i th row and j th column
                ro.append(self.table.find_element_by_xpath("//tr["+str(i)+"]/td["+str(j)+"]").text);

            # add the row data to allData of the self.table
            allData.append(ro);

        return allData;

    def presence_of_data(self, data):

        # verify the data by getting the size of the element matches based on the text/data passed
        dataSize = len(self.table.find_elements_by_xpath("//td[normalize-space(text())='"+data+"']"))
        presence = false;
        if(dataSize > 0):
            presence = true;
        return presence;

    def get_cell_data(self, row_number, column_number):
        if(rowNumber == 0):
            raise Exception("Row number starts from 1");

        rowNumber = rowNumber+1;
        cellData = table.find_element_by_xpath("//tr["+str(row_number)+"]/td["+str(column_number)+"]").text;
        return cellData;

In [6]:
def getString(length=3, characters='abcdefghijklmnopqrstuvwxyz'):
    for s in itertools.product(characters, repeat=length):
        yield ''.join(s)

In [7]:
def join_db(db):
    cac_db = pd.read_csv('data\cac_db.csv')
    cac_db_main = pd.concat([cac_db, db])
    cac_db_main = cac_db_main.drop_duplicates()
    cac_db_main.reset_index(drop=True, inplace=True)
    cac_db_main.to_csv('data\cac_db.csv', index = None, header=True)

In [8]:
def cac_scraper(x):
    """Scrapes CAC website for company info
    args:
    x: str, letter combination to continue scraping from
    """
    driver = webdriver.Chrome("data\chromedriver.exe")
    string = [s for s in getString()]
    idx = string.index(x)
    url = 'http://publicsearch.cac.gov.ng/comsearch/'
    driver.get(url)
    time.sleep(10)
    data=[]
    for s in string[idx:]:
        try:
            driver.find_element_by_css_selector('input.field').send_keys(s)
            element = WebDriverWait(driver, 300).until(EC.presence_of_element_located((By.ID, "directorsTbl")))

            table = driver.find_element_by_id('directorsTbl')
            if len(table.find_elements_by_tag_name("tr")) >= 1:
                CACtable = WebTable(table)
                db = CACtable.get_all_data()
                data.append(db)
            else:
                pass
            driver.refresh()
        except:
            break
    cac_data=[]
    for _ in data:
        for d in _:
            cac_data.append(d) 
    col_names = ['RC NUMBER', 'COMPANY NAME', 'ADDRESS']
    cac_db = pd.DataFrame(cac_data, columns=col_names)
    print(f"database creation stopped at {s}")
    join_db(cac_db)

In [17]:
#cac_scraper('aao')
cac_db = pd.read_csv('https://raw.githubusercontent.com/chocolatebunny-0/Task7-isthisarealjob/master/data/cac_db.csv')

In [ ]:
df= pd.read_csv("https://raw.githubusercontent.com/chocolatebunny-0/Task7-isthisarealjob/master/data/cac_db.csv")
Company = input('')
if df['COMPANY NAME'].str.contains(Company).any():
    print("Company is in CAC database")
else:
   print("Company is not registered in CAC database")